In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer 
import time
import datetime
import json

from tqdm import tqdm

from datasets import load_dataset 
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer
import numpy as np
import pyarrow as pa
import evaluate
import torch.nn as nn

In [2]:
# dataset = load_dataset("yelp_review_full")
    # print(type(dataset))
    # print(type(dataset["train"]))
    # print(type(dataset["train"][0]))
    # print((dataset["train"][0]))
    # print()
print("[Loading tokenizer...]")
dataset = load_dataset("json", data_files = "kick_train.json")
print("[Loading tokenizer complete]")


    # tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print("[Loading tokenizer...]")
tokenizer = AutoTokenizer.from_pretrained(
        "KT-AI/midm-bitext-S-7B-inst-v1",
        trust_remote_code = True
    )
print("[Loading tokenizer complete]")

[Loading tokenizer...]
[Loading tokenizer complete]
[Loading tokenizer...]
[Loading tokenizer complete]


In [3]:
# dummy_data = "### User; I see a korean report of a chemical industry, and some text field says '개스킷 교체 필요함' in a container of a chemical container. What does this mean, and what should I do? \n ### Midm;"
    
# start = time.time()
# data = tokenizer(dummy_data, return_tensors = "pt")
# end = time.time()
# sec = (end - start)
# result = datetime.timedelta(seconds = sec)
# print("Tokenizing Time: ", result)

# print(data)

def tokenize_function(examples):
    # return tokenizer(examples["text"], padding = "max_length", truncation = True)
    return tokenizer(examples["input"], return_tensors = "pt", padding = "max_length", truncation = True, max_length=697)

print("[Tokenizing dataset...]")
tokenized_dataset = dataset.map(tokenize_function, batched = True)
print("[Tokenizing dataset complete]")

[Tokenizing dataset...]


Map:   0%|          | 0/76579 [00:00<?, ? examples/s]

[Tokenizing dataset complete]


In [4]:
print("[Sampling dataset...]")
# train_dataset = tokenized_dataset["train"].shuffle(seed = 42).select( range( int(len(tokenized_dataset["train"]) * 0.9) ) )
# eval_range = range( int(len(tokenized_dataset["train"]) * 0.1) - 1 )
# eval_range = [ i + int(len(tokenized_dataset["train"]) * 0.9) for i in eval_range ]
# eval_dataset = tokenized_dataset["train"].shuffle(seed = 42).select( eval_range )
train_dataset = tokenized_dataset["train"].shuffle(seed = 42).select( range( 10000 ) )
eval_range = [ i + 10000 for i in range(1000) ]
eval_dataset = tokenized_dataset["train"].shuffle(seed = 42).select( eval_range )
print("[Sampling dataset complete]")

print("\t", type(train_dataset))
print("\t", train_dataset.shape)
print("\t", type(eval_dataset))
print("\t", eval_dataset.shape)

[Sampling dataset...]
[Sampling dataset complete]
	 <class 'datasets.arrow_dataset.Dataset'>
	 (10000, 5)
	 <class 'datasets.arrow_dataset.Dataset'>
	 (1000, 5)


In [5]:
    # model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
print("[Loading model...]")
model = AutoModelForCausalLM.from_pretrained(
        "KT-AI/midm-bitext-S-7B-inst-v1", 
        trust_remote_code = True
    )
print("[Loading model complete]")

print("[Model cuda...]")
model.cuda()
print("[Model cuda complete]")

print("[Model train...]")
model.train()
print("[Model train complete]")

[Loading model...]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[Loading model complete]
[Model cuda...]
[Model cuda complete]
[Model train...]
[Model train complete]


In [7]:
    # training_args = TrainingArguments(output_dir = "/root/fine_tuned_midm_checkpoints")
training_args = TrainingArguments(
    output_dir = "/root/fine_tuned_midm_checkpoints", 
    evaluation_strategy = "epoch",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1
)

# Evaluate
    # metric = evaluate.load("accuracy")
    # def compute_metrics(eval_pred):
    #     logits, labels = eval_pred
    #     predictions = np.argmax(logits, axis = -1)
        
    #     return metric.compute(predictions = predictions, references=labels)
loss_fn = nn.CrossEntropyLoss(reduction = "sum")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    loss = loss_fn(logits, labels)
    return loss

In [9]:
print("[Trainer definition...]")
trainer = Trainer(
    model = model,
    args = training_args,
    
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics
)
print("[Trainer definition complete]")

[Trainer definition...]
[Trainer definition complete]


In [10]:
print("[Training...]")
trainer.train()
print("[Training complete]")

exit()

[Training...]


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [48]:
model.eval()
anrg="""1300년도에 세종대왕이 그리스를 방문했던 사건이 뭐였지?"""
dummy_data = f"###User;{anrg}\n###Midm;"
data = tokenizer(dummy_data, return_tensors="pt")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
pred = model.generate(
    input_ids=data.input_ids[..., :-1].cuda(),
    streamer=streamer,
    use_cache=True,
    max_new_tokens=float('inf')
)
decoded_text = tokenizer.batch_decode(pred[0], skip_special_tokens=True)
model.train()
1

죄송하지만, 제가 제공할 수 있는 정보가 없습니다. 1300년도에 세종대왕이 그리스를 방문했다는 정보는 없습니다. 더 자세한 정보를 제공해 주실 수 있나요?


1